In [3]:
import json
import os
from time import time
import urllib.parse

import polars as pl
from tqdm import tqdm

In [4]:
source_path = '/home/matthew/data/01-F-parquet-duplicated/P5MIN_UNITSOLUTION.parquet'
dest_path = '/home/matthew/data/01-G-parquet-deduplicated/P5MIN_UNITSOLUTION.parquet'
dest_path_gb = '/home/matthew/data/01-G-parquet-deduplicated/P5MIN_UNITSOLUTION-gb.parquet'
schema_path = '/home/matthew/Documents/TSE/AppliedEconometrics/repo/data/schemas.json'

In [5]:
with open(schema_path, 'r') as f:
    schemas = json.load(f)

In [6]:
schema = schemas['P5MIN_UNITSOLUTION']
sort_keys = ['SCHEMA_VERSION', 'LASTCHANGED', 'TOP_TIMESTAMP']
primary_keys = ["DUID", "INTERVAL_DATETIME", "RUN_DATETIME"]

In [7]:
# on Linux and Mac this makes this python process lower priority
# so when it's running and using up all your CPU, your interface won't lag.
# So you can keep browsing the web, typing documents etc
try:
    os.nice(20)
except OSError: # ignore error, this is probably on Windows
    pass

In [5]:
# # doesn't work
# q = (
#     pl.scan_parquet(source_path, low_memory=True).
#     with_columns(
#         pl.struct(*sort_keys)
#         .rank("dense", descending=True)
#         .over(*primary_keys)
#         .alias("Rank")
#     )
#     .filter(pl.struct(*primary_keys).is_duplicated())
#     .select(pl.exclude("SCHEMA_VERSION", "TOP_TIMESTAMP"))
#     .sink_parquet(dest_path)
# )
# print(q)

In [ ]:
t1 = time()
(
    pl.scan_parquet(source_path, low_memory=True)
    .sort(sort_keys, descending=True)
    .unique(primary_keys)
    .select(pl.exclude("SCHEMA_VERSION", "TOP_TIMESTAMP"))
    .sink_parquet(dest_path)
)
t2 = time()
t_delta = t2 - t1
print(t_delta)

In [6]:
t3 = time()
(
    pl.scan_parquet(source_path, low_memory=True)
    .sort(sort_keys, descending=True)
    .group_by(primary_keys)
    .first()
    .select(pl.exclude("SCHEMA_VERSION", "TOP_TIMESTAMP"))
    .sink_parquet(dest_path_gb)
)
t4 = time()
t_delta_b = t4 - t3
print(t_delta_b)

NameError: name 't2' is not defined

In [ ]:
paths = []
parent = '/home/matthew/data/01-G-parquet-deduplicated/P5MIN_UNITSOLUTION/'
for subdir in os.listdir(parent):
    for sub_subdir in os.listdir(os.path.join(parent, subdir)):
        paths.append(os.path.join(parent, subdir, sub_subdir))
pl.scan_parquet(paths).sink_parquet('/home/matthew/data/01-G-parquet-deduplicated/P5MIN_UNITSOLUTION-recombined.parquet')


In [40]:
duids = ds.select('DUID').filter(pl.col("DUID").is_first_distinct()).collect()['DUID']

In [41]:
base_dir = '/home/matthew/data/01-G-parquet-deduplicated/P5MIN_UNITSOLUTION/'


for duid in tqdm(duids):
    dest_path = f'{base_dir}/DUID={urllib.parse.quote_plus(duid)}/data.parquet'
    if not os.path.exists(os.path.dirname(dest_path)):
        os.makedirs(os.path.dirname(dest_path))
    ds.filter(
        pl.col("DUID").eq(duid),
    ).select(pl.exclude("DUID")).sink_parquet(dest_path)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 243/243 [45:58<00:00, 11.35s/it]
